In [1]:
!pip install catboost

import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

     |████████████████████████████████| 65.8MB 71kB/s 
Mounted at /content/drive
     |████████████████████████████████| 296kB 13.1MB/s 
     |████████████████████████████████| 1.1MB 22.2MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 481kB 28.5MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 143kB 57.1MB/s 
     |████████████████████████████████| 112kB 53.3MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=aa598324532b1e63c9275cda04e28c398aeb83dca1217e187fdddbae1fd260c4
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=0c4e54a38c28400c26e7fe8b3ebedc6340fc50f5b0547f177f00a1ffc2e6ea62
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created

In [2]:
from sklearn.metrics import mean_squared_error

In [3]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [4]:
cat_features = [feature for feature in train_data.columns if 'cat' in feature]
cont_features = [feature for feature in train_data.columns if 'cont' in feature]
for feature in cat_features:
    train_data[feature] = train_data[feature].astype('category')
    test_data[feature] = test_data[feature].astype('category')

In [9]:
def objective_CAT(trial: Trial, X, y):

  params = {} #initialize parameters
  params['random_state'] = 42
  params['iterations'] = 500
  params['subsample'] = trial.suggest_loguniform('subsample', 0.3, 1)
  params['max_depth'] = trial.suggest_int('max_depth', 3, 10)
  params['l2_leaf_reg'] = trial.suggest_loguniform('l2_leaf_reg', 0.3, 100)
  params['objective'] = trial.suggest_categorical('objective', ['MAE', 'MAPE', 'Poisson', 'RMSE'])
  params['bootstrap_type'] = trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS', 'Poisson'])
  params['min_data_in_leaf'] = trial.suggest_int('min_data_in_leaf', 1, 200)
  # params['max_leaves'] = trial.suggest_int('max_leaves', 20, 200)
  params['cat_features'] = cat_features
  params['task_type'] = 'GPU'
  params['max_bin'] = 36
  params['learning_rate'] = 0.01


  # train_data = X.iloc[:, :-1]
  # train_target = X.iloc[:,-1]
  X_train, X_valid, Y_train,  Y_valid = train_test_split(X, y, test_size=0.20, random_state=42)
  # print('#############################    BEGIN   #######################################')
  model = catboost.CatBoostRegressor(**params)
  model.fit(X_train, Y_train,eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=False)
  # print('#############################    END   #######################################')

  pred = model.predict(train_data)
  # score = cv_evaluation_regression(X, model)
  score = mean_squared_error(pred, train_target)

  return score

In [10]:
study = optuna.create_study(direction = 'minimize')
study.optimize(lambda trial : objective_CAT(trial, train_data, train_target), n_trials=200)

[I 2021-02-11 14:12:22,479] A new study created in memory with name: no-name-58a1221e-66c7-4047-96d8-d101fb069884
[I 2021-02-11 14:12:39,547] Trial 0 finished with value: 0.7902204452569493 and parameters: {'subsample': 0.5082696741792726, 'max_depth': 9, 'l2_leaf_reg': 12.044556297864183, 'objective': 'MAPE', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 141}. Best is trial 0 with value: 0.7902204452569493.
[I 2021-02-11 14:12:54,303] Trial 1 finished with value: 0.79881175658621 and parameters: {'subsample': 0.5902417978315306, 'max_depth': 7, 'l2_leaf_reg': 2.462268350464739, 'objective': 'MAPE', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 25}. Best is trial 0 with value: 0.7902204452569493.
[I 2021-02-11 14:13:07,274] Trial 2 finished with value: 0.746300176580536 and parameters: {'subsample': 0.5003940523955505, 'max_depth': 5, 'l2_leaf_reg': 20.00710127448401, 'objective': 'RMSE', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 62}. Best is trial 2 with value: 0.74630017

In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
study.best_params

{'bootstrap_type': 'Bernoulli',
 'l2_leaf_reg': 0.5925500443911146,
 'max_depth': 10,
 'min_data_in_leaf': 95,
 'objective': 'RMSE',
 'subsample': 0.9965285997680801}